### Project idea

### Import modules

In [1]:
import os
import tensorflow as tf
import tfx
from tfx import v1 as tfx
from tfx.components import *
from tfx.proto import *
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2

print('Tensorflow Version : {}'.format(tf.__version__))
print('Tensorflow Extended Version : {}'.format(tfx.__version__))

Tensorflow Version : 2.9.1
Tensorflow Extended Version : 1.9.1


### Global declaration

In [2]:
project_dir = os.getcwd()
production_model_dir = os.path.join(project_dir, 'prod-models')
data_root = os.path.join(project_dir, 'dataset')
data_file = os.path.join(project_dir, 'dataset','titanic.csv')

transform_module = os.path.join(project_dir,'modules','transform_module.py')
trainer_module = os.path.join(project_dir,'modules','trainer_module.py')

cache = True
context = InteractiveContext()

### Example Gen Component

In [3]:
example_gen_com = CsvExampleGen(input_base=data_root)
context.run(example_gen_com, enable_cache=cache)

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Statistics Gen Component

In [4]:
statistics_gen_com = StatisticsGen(examples=example_gen_com.outputs['examples'])
context.run(statistics_gen_com, enable_cache=cache)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [5]:
context.show(statistics_gen_com.outputs['statistics'])

### Schema Gen Component

In [6]:
schema_gen_com = SchemaGen(statistics=statistics_gen_com.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen_com, enable_cache=cache)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [7]:
context.show(schema_gen_com.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',FLOAT,required,,-
'Cabin',BYTES,required,,-
'Embarked',STRING,required,,'Embarked'
'Fare',FLOAT,required,single,-
'Name',BYTES,required,single,-
'Parch',INT,required,single,-
'PassengerId',INT,required,single,-
'Pclass',INT,required,single,-
'Sex',STRING,required,single,'Sex'


,Values
Domain,
'Embarked',"'C', 'Q', 'S'"
'Sex',"'female', 'male'"


### ExampleValidator Component

In [8]:
example_validator_com = ExampleValidator(schema=schema_gen_com.outputs['schema'], statistics=statistics_gen_com.outputs['statistics'])
context.run(example_validator_com, enable_cache=cache)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [9]:
context.show(example_validator_com.outputs['anomalies'])

### Transform Component

In [10]:
transform_com = Transform(examples=example_gen_com.outputs['examples'], schema=schema_gen_com.outputs['schema'], module_file=transform_module)
context.run(transform_com, enable_cache=cache)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /tmp/tmpoiqhzi1k
running install
running install_lib
copying build/lib/trainer_module.py -> /tmp/tmpoiqhzi1k
copying build/lib/transform_module.py -> /tmp/tmpoiqhzi1k
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpoiqhzi1k/tfx_user_code_Transform-0.0+288560fdec297a5c39e9340d75427fd0fc79cb8faaf6c08eb794475e331c26d9-py3

Processing /tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/_wheels/tfx_user_code_Transform-0.0+288560fdec297a5c39e9340d75427fd0fc79cb8faaf6c08eb794475e331c26d9-py3-none-any.whl


Processing /tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/_wheels/tfx_user_code_Transform-0.0+288560fdec297a5c39e9340d75427fd0fc79cb8faaf6c08eb794475e331c26d9-py3-none-any.whl


Processing /tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/_wheels/tfx_user_code_Transform-0.0+288560fdec297a5c39e9340d75427fd0fc79cb8faaf6c08eb794475e331c26d9-py3-none-any.whl


2022-08-19 17:50:30.287890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

### Trainer Component

In [11]:
train_com_input = {
    'module_file': trainer_module,
    'examples': transform_com.outputs['transformed_examples'],
    'transform_graph': transform_com.outputs['transform_graph'],
    'train_args': trainer_pb2.TrainArgs(num_steps=100),
    'eval_args': trainer_pb2.EvalArgs(num_steps=1),
}

trainer_com = Trainer(**train_com_input)
context.run(trainer_com, enable_cache=cache)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /tmp/tmpq0vrut3r
running install
running install_lib
copying build/lib/trainer_module.py -> /tmp/tmpq0vrut3r
copying build/lib/transform_module.py -> /tmp/tmpq0vrut3r
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpq0vrut3r/tfx_user_code_Trainer-0.0+288560fdec297a5c39e9340d75427fd0fc79cb8faaf6c08eb794475e331c26d9-py3.8.egg-info
runnin

Processing /tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/_wheels/tfx_user_code_Trainer-0.0+288560fdec297a5c39e9340d75427fd0fc79cb8faaf6c08eb794475e331c26d9-py3-none-any.whl


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Age_xf (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 Cabin_xf (InputLayer)          [(None, 114)]        0           []                               
                                                                                                  
 Embarked_xf (InputLayer)       [(None, 13)]         0           []                               
                                                                                                  
 Fare_xf (InputLayer)           [(None, 1)]          0           []                               
                                                                                              

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


/tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/Trainer/model/6/Format-Serving
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-08-19T17_50_13.658286-qak1cxif/Trainer/model/6/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [12]:
model_run_artifact_dir = trainer_com.outputs['model_run'].get()[0].uri
%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir} --port=9002 --host 0.0.0.0

### Evaluator Component

In [13]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="serving_default",
            label_key="Survived",
            # preprocessing_function_names=["transform_features"],
        )
    ],
    # slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=["product"])],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.65}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": -1e-10},
                        ),
                    ),
                ),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
            ],
        )
    ],
)

evaluator_com = Evaluator(
    examples=example_gen_com.outputs["examples"],
    model=trainer_com.outputs["model"],
    eval_config=eval_config,
)
context.run(evaluator_com, enable_cache=cache)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [14]:
context.show(evaluator_com.outputs['evaluation'])

### Pusher Component

In [15]:
pusher_com = Pusher(
    model=trainer_com.outputs["model"],
    model_blessing=evaluator_com.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=production_model_dir
        )
    ),
)
context.run(pusher_com, enable_cache=cache)

ExecutionResult(
    component_id: Pusher
    execution_id: 8
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))